In [3]:
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from dc_qiskit_stochastics.simulation import lognormal

c:\users\cncen\.virtualenvs\dc-qiskit-stochastics-i_-wdnbc\lib\site-packages\qiskit\quantum_info\operators\pauli.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z = np.zeros(len(label), dtype=np.bool)
c:\users\cncen\.virtualenvs\dc-qiskit-stochastics-i_-wdnbc\lib\site-packages\qiskit\quantum_info\operators\pauli.py:112: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-no

In [4]:
s0 = 50
sigma = np.sqrt(0.01)
mu = 0.02
q = np.linspace(0.05, 0.95, 8)

In [23]:
def compute_states(t: float):
    s = sigma * np.sqrt(t)
    mu_tilde = (mu - sigma**2 / 2) * t + np.log(s0)
    scale = np.exp(mu_tilde)
    xq = lognorm.ppf(q, s=s, scale=scale)
    return xq


def compute_probability(current_state: float, target_states: np.ndarray, delta_t: float):

    s = sigma * np.sqrt(delta_t)
    mu_tilde = (mu - sigma**2 / 2) * delta_t + np.log(current_state)
    scale = np.exp(mu_tilde)

    _p = lognorm.pdf(target_states, s=s, scale=scale)
    p = _p / np.sum(_p)
    return p


In [34]:
delta_t = 3


states = {}
transition_matrices = {}
last_states = np.asarray([s0])
for level in range(1, 5):
    target_states = compute_states(level * delta_t)

    transition_matrix = np.zeros(shape=(last_states.shape[0], target_states.shape[0]))
    for row, cs in enumerate(last_states):
        transition_matrix[row, :] = compute_probability(cs, target_states, delta_t)

    transition_matrices[level] = transition_matrix
    states[level] = last_states
    last_states = target_states


In [35]:
states

{1: array([50]),
 2: array([39.33544419, 44.59098852, 47.92961801, 50.85562691, 53.78826052,
        57.07192803, 61.34503406, 69.54124367]),
 3: array([36.5659616 , 43.66157226, 48.35536636, 52.58229377, 56.92112673,
        61.89682   , 68.55097636, 81.85326673]),
 4: array([34.93771544, 43.41374572, 49.19703686, 54.5146654 , 60.07394713,
        66.56724339, 75.43488987, 93.73569753])}

In [36]:
transition_matrices

{1: array([[0.06130838, 0.13691017, 0.17141545, 0.18104159, 0.17117087,
         0.14395653, 0.09951841, 0.03467859]]),
 2: array([[3.19402360e-01, 3.18145257e-01, 1.97285047e-01, 1.02801649e-01,
         4.47261698e-02, 1.47779388e-02, 2.79381844e-03, 6.77594541e-05],
        [1.25284925e-01, 2.61911473e-01, 2.48882969e-01, 1.84093554e-01,
         1.11568298e-01, 5.23275561e-02, 1.51595639e-02, 7.71661442e-04],
        [6.14401777e-02, 1.96824931e-01, 2.39137608e-01, 2.16413229e-01,
         1.58730077e-01, 9.10838529e-02, 3.37384221e-02, 2.63170253e-03],
        [3.12293269e-02, 1.42012888e-01, 2.11099912e-01, 2.25430386e-01,
         1.93376250e-01, 1.30940434e-01, 5.93402911e-02, 6.57051171e-03],
        [1.52830623e-02, 9.68090854e-02, 1.74160478e-01, 2.17515027e-01,
         2.16386750e-01, 1.71362806e-01, 9.39865387e-02, 1.44962536e-02],
        [6.64288604e-03, 5.97310333e-02, 1.31469811e-01, 1.93755560e-01,
         2.25429543e-01, 2.10661517e-01, 1.41360093e-01, 3.09495567e-